In [ ]:
%pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from trl import SFTConfig, SFTTrainer, setup_chat_format
from datasets import load_dataset
from datasets import Dataset

checkpoint = "HuggingFaceTB/SmolLM-135M-Instruct"
device = "cpu"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# for multiple GPUs install accelerate and do
model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")
# model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
ds = load_dataset("medalpaca/medical_meadow_medical_flashcards")
# ds = load_dataset("bala1524/Medical-QA-Mistral7B-Finetuning")

README.md:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

(…)l_meadow_wikidoc_medical_flashcards.json:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/33955 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset
from datasets import Dataset
import pandas as pd

pandas_data = pd.DataFrame(ds['train'])
pandas_data = pandas_data[:100]
# pandas_data.dropna(inplace=True)
ds = Dataset.from_pandas(pandas_data)

In [ ]:
def tokenize_function(examples):
    examples["text"] = tokenizer.apply_chat_template([{"role": "user", "content": examples["input"]}, {"role": "assistant", "content": examples["output"]}], tokenize=False)
    return examples
ds = ds.map(tokenize_function)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
sft_config = SFTConfig(
    output_dir="./sft_output",
    num_train_epochs=10,
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=100,  # Frequency of logging training metrics
    use_mps_device= True if device == "mps" else False,
    # hub_model_id="argilla/SmolLM2-360M-synthetic-concise-reasoning2",  # Set a unique name for your model
    push_to_hub=False,
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds,
    tokenizer=tokenizer,
)
trainer.train()

<ipython-input-6-d3ca632c585b>:13: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Converting train dataset to ChatML:   0%|          | 0/100 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kl-kaczorowski (kl-kaczorowski-sgh-warsaw-school-of-economics) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
100,1.029000
200,0.267700


TrainOutput(global_step=250, training_loss=0.5490079288482665, metrics={'train_runtime': 1856.5399, 'train_samples_per_second': 0.539, 'train_steps_per_second': 0.135, 'total_flos': 46473782717952.0, 'train_loss': 0.5490079288482665})

In [ ]:
# prompt = "What is the relationship between very low Mg2+ levels, PTH levels, and Ca2+ levels?"

prompt = "Low REM sleep latency"

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
pipe(prompt, max_new_tokens=30)

Device set to use cpu


[{'generated_text': 'Low REM sleep latency and duration\n\t+ Duration of REM sleep episodes\n\t+ REM sleep stage characteristics (e.g., rapid eye movements, sleep spindles'}]